In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
import numpy as np
from sklearn.metrics import silhouette_score,davies_bouldin_score
import re
from sklearn.mixture import BayesianGaussianMixture

In [ ]:
df1=pd.read_csv('../../data/dataset.csv')
mask=df1['F']==3
df2=df1[~mask].copy()
df=df2[['Bandgap, GGA (eV)', 'Dielectric constant, total', 'Refractive index',
       'Atomization energy (eV/atom)', 'Relative energy1 (eV/atom)',
       'Relative energy2 (eV/atom)', 'Volume of the unit cell (A^3)',
       'Density (g/cm^3)', 'Tolerance factor', 'Octahedral factor', 'rA(Ang)',
       'rB(Ang)', 'rX(Ang)', 'A SITE DFE',
       'B SITE DFE', 'X SITE DFE', 'a', 'b', 'c', 'alpha',
       'beta', 'gamma']]


In [ ]:
halide_groups = df1['Label'].apply(lambda x: 'Chloride' if 'Chloride' in x else 'Bromide' if 'Bromide' in x else 'Iodide')
df['Halide Group'] = halide_groups

# Example: Create box plots for defect energies grouped by cation/anion families
cation_families = df1['Label'].apply(lambda x: 'Germanium' if 'Germanium' in x else 'Tin' if 'Tin' in x else 'Lead')
df['Cation Family'] = cation_families

patterns = {
    'Ammonium': r'Ammonium',
    'Methylammonium': r'Methylammonium',
    'Dimethylammonium': r'Dimethylammonium',
    'Trimethylammonium': r'Trimethylammonium',
    'Tetramethylammonium': r'Tetramethylammonium',
    'Ethylammonium': r'Ethylammonium',
    'Propylammonium': r'Propylammonium',
    'Isopropylammonium': r'Isopropylammonium',
    'Butylammonium': r'Butylammonium',
    'Hydroxylammonium': r'Hydroxylammonium',
    'Formamidinium': r'Formamidinium',
    'Acetamidinium': r'Acetamidinium',
    'Hydrazinium': r'Hydrazinium',
    'Guanidinium': r'Guanidinium',
    'Azetidinium': r'Azetidinium',
    'Imidazolium': r'Imidazolium'
}

# Function to map the Label to the ammonium family using regex
def map_ammonium_family(label):
    for family, pattern in patterns.items():
        if re.search(pattern, label):
            return family
    return 'Unknown'  # Default if no match is found

# Apply the mapping function to the 'Label' column
df['a_site_family'] = df1['Label'].apply(map_ammonium_family)

# Select numerical features for t-SNE
data = df.drop([ 'Halide Group','Cation Family','a_site_family'], axis=1)
print(data.shape, data.columns)


# Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data)

In [ ]:
import optuna
import numpy as np
from sklearn.manifold import TSNE
from sklearn.mixture import BayesianGaussianMixture
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
import plotly.express as px

def multi_objective(trial):
    # t-SNE parameters
    tsne_perplexity = trial.suggest_int('tsne_perplexity', 5, 50)
    # Suggest the mode for learning rate: either 'auto' or 'int'
    tsne_lr_mode = trial.suggest_categorical("tsne_lr_mode", ["auto", "int"])

    if tsne_lr_mode == "auto":
        tsne_learning_rate = "auto"
    else:
    # Suggest an integer learning rate between 10 and 1000 on a logarithmic scale
        tsne_learning_rate = trial.suggest_int("tsne_learning_rate", 10, 1000, log=True)
    # tsne_learning_rate = trial.suggest_int('tsne_learning_rate', 10, 1000, log=True)
    
    tsne_n_iter = trial.suggest_int('tsne_n_iter', 500, 4000)
    
    # BGMM parameters
    bgmm_n_components = trial.suggest_int('bgmm_n_components', 2, 10)
    # bgmm_covariance_type = 'full'
    # bgmm_init = trial.suggest_categorical('bgmm_init', ['kmeans', 'k-means++', 'random', 'random_from_data'])
    
    try:
        # t-SNE transformation
        tsne = TSNE(n_components=2,
                    perplexity=tsne_perplexity,
                    learning_rate=tsne_learning_rate,
                    n_iter=tsne_n_iter,
                    random_state=42)
        tsne_results = tsne.fit_transform(scaled_data)
        
        # BGMM clustering
        bgmm = BayesianGaussianMixture(
            n_components=bgmm_n_components,
            covariance_type='full',
            init_params='k-means++',
            max_iter=1000,
            random_state=42
        )
        labels = bgmm.fit_predict(tsne_results)
        
        # Check cluster validity
        unique_labels = np.unique(labels)
        if len(unique_labels) < 2:
            return np.inf, np.inf, -np.inf  # Worst possible scores
        
        # Calculate metrics
        silhouette = silhouette_score(tsne_results, labels)
        dbi = davies_bouldin_score(tsne_results, labels)
        calinski = calinski_harabasz_score(tsne_results, labels)
        
        return dbi, -silhouette, -calinski  # We minimize all (convert max problems to min)
    
    except Exception as e:
        print(f"Trial {trial.number} failed: {str(e)}")
        return np.inf, np.inf, np.inf

# Create multi-objective study
study = optuna.create_study(
    directions=['minimize', 'minimize', 'minimize'],  # DBI ↓, Silhouette↑ (via -silhouette), Calinski↑ (via -calinski)
    sampler=optuna.samplers.NSGAIISampler(population_size=50),
    study_name='tsne_bgmm_multiobj'
)

# Run optimization
study.optimize(multi_objective, n_trials=1000, show_progress_bar=True)

fig = optuna.visualization.plot_pareto_front(
    study,
    targets=lambda t: (t.values[0], -t.values[1], -t.values[2]),
    target_names=["DBI", "Silhouette", "Calinski"],
    include_dominated_trials=False
)
fig.update_layout(title='3D Pareto Front of Clustering Metrics')
fig.show()

pareto_trials = study.best_trials

print(f"Found {len(pareto_trials)} Pareto-optimal solutions:")
for i, trial in enumerate(pareto_trials):
    print(f"\nSolution {i+1}:")
    print(f"  DBI: {trial.values[0]:.3f}")
    print(f"  Silhouette: {-trial.values[1]:.3f}")
    print(f"  Calinski: {-trial.values[2]:.3f}")
    print("  Parameters:")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")

for i, metric in enumerate(["DBI", "Silhouette", "Calinski"]):
    fig = optuna.visualization.plot_param_importances(
        study,
        target=lambda t: t.values[i],
        target_name=metric
    )
    fig.update_layout(title=f'Parameter Importance for {metric}')
    fig.show()


In [ ]:
# # Apply t-SNE
tsne = TSNE(n_components=2, perplexity=49, n_iter=1093, learning_rate=202, random_state=42)
tsne_results = tsne.fit_transform(scaled_data)
# Create a DataFrame with t-SNE results
tsne_df = pd.DataFrame(data=tsne_results, columns=['TSNE1', 'TSNE2'])

In [ ]:
import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from sklearn.mixture import BayesianGaussianMixture
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.datasets import make_blobs

optimal_gmm = BayesianGaussianMixture(n_components=4, covariance_type='full',
                                      init_params='k-means++',  
                                      max_iter=1000, random_state=42)
optimal_gmm.fit(tsne_df[['TSNE1', 'TSNE2']])

# Get the predicted labels
optimal_labels = optimal_gmm.predict(tsne_df[['TSNE1', 'TSNE2']])

# Compute silhouette scores for each sample
sample_silhouette_values = silhouette_samples(tsne_df[['TSNE1', 'TSNE2']], optimal_labels)

# --- Figure 2: Silhouette Analysis for Optimal Clusters ---
fig, ax1 = plt.subplots(figsize=(12, 7))  # Create a new figure
ax1.set_xlim([-0.1, 1])
ax1.set_ylim([0, len(tsne_df) + (optimal_silhouette + 1) * 10])

# Compute the silhouette score for the entire dataset
silhouette_avg = silhouette_score(tsne_df[['TSNE1', 'TSNE2']], optimal_labels)
ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

y_lower = 10
for i in range(optimal_silhouette):
    ith_cluster_silhouette_values = sample_silhouette_values[optimal_labels == i]
    ith_cluster_silhouette_values.sort()

    size_cluster_i = ith_cluster_silhouette_values.shape[0]
    y_upper = y_lower + size_cluster_i

    color = cm.nipy_spectral(float(i) / optimal_silhouette)
    ax1.fill_betweenx(
        np.arange(y_lower, y_upper),
        0,
        ith_cluster_silhouette_values,
        facecolor=color,
        edgecolor=color,
        alpha=0.7,
    )
    
    ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i), fontsize=18, fontweight='bold')
    y_lower = y_upper + 10

# Set font properties for ax1
# ax1.set_title("Silhouette Plot for the Optimal Number of Clusters", fontsize=24, fontweight='bold')
ax1.set_xlabel("Silhouette Coefficient Values", fontsize=20, fontweight='bold')
ax1.set_ylabel("Cluster Label", fontsize=20, fontweight='bold')
ax1.set_yticks([])  # Clear the yaxis labels / ticks

# Explicitly set bold tick labels for ax1
ax1.set_xticklabels(ax1.get_xticklabels(), fontsize=18, fontweight='bold')
ax1.set_yticklabels(ax1.get_yticklabels(), fontsize=18, fontweight='bold')

# Set the ticks to be bold
ax1.tick_params(axis='both', labelsize=18, width=2)

# Make the box outlines bold for ax1
ax1.spines['top'].set_linewidth(2)
ax1.spines['right'].set_linewidth(2)
ax1.spines['left'].set_linewidth(2)
ax1.spines['bottom'].set_linewidth(2)

# Save the figure as PDF
plt.savefig("silhouette_analysis.pdf", bbox_inches='tight', dpi=2000, transparent=True)

plt.show()

# --- Figure 3: Visualization of the Clusters ---
fig, ax2 = plt.subplots(figsize=(12, 7))  # Create a new figure

# Visualizing the actual clusters formed
colors = cm.nipy_spectral(optimal_labels.astype(float) / optimal_silhouette)
ax2.scatter(
    tsne_df['TSNE1'], tsne_df['TSNE2'], marker=".", s=100, lw=0, alpha=0.7, c=colors, edgecolor="k"
)

# Labeling the clusters
centers = optimal_gmm.means_
ax2.scatter(
    centers[:, 0],
    centers[:, 1],
    marker="o",
    c="white",
    alpha=1,
    s=700,
    edgecolor="k",
    linewidth=2
)

# Using ax.text to add the cluster index labels at the centers
for i, c in enumerate(centers):
    ax2.text(c[0], c[1], f"${i}$", fontsize=18, fontweight='bold', color='black', ha='center', va='center')

# ax2.set_title("Visualisation of Clusters", fontsize=24, fontweight='bold')
ax2.set_xlabel("TSNE1", fontsize=20, fontweight='bold')
ax2.set_ylabel("TSNE2", fontsize=20, fontweight='bold')

# Explicitly set bold tick labels for ax2
ax2.set_xticklabels(ax2.get_xticklabels(), fontsize=18, fontweight='bold')
ax2.set_yticklabels(ax2.get_yticklabels(), fontsize=18, fontweight='bold')

# Set ticks and font weight for ax2
ax2.tick_params(axis='both', labelsize=18, width=2)

# Make the box outlines bold for ax2
ax2.spines['top'].set_linewidth(2)
ax2.spines['right'].set_linewidth(2)
ax2.spines['left'].set_linewidth(2)
ax2.spines['bottom'].set_linewidth(2)

# Save the figure as PDF
plt.savefig("cluster_visualization.pdf", bbox_inches='tight', dpi=2000, transparent=True)

plt.show()


In [ ]:
gmm = BayesianGaussianMixture(n_components=4, covariance_type='full', 
                                  init_params='k-means++', 
                                  # weight_concentration_prior_type='dirichlet_process',
                              max_iter=1000, random_state=42)

clusters = gmm.fit_predict(tsne_df[['TSNE1', 'TSNE2']])  # Numerical cluster labels
tsne_df['Cluster'] = clusters  # Assign the cluster labels
print(tsne_df['Cluster'])

data['Cluster'] = clusters

cluster_profile = data.groupby('Cluster').mean()
data = data.reset_index(drop=True)
data['a_site_family']=df['a_site_family'].reset_index(drop=True)
data['Halide Group']=df['Halide Group'].reset_index(drop=True)
data['Cation Family'] = df['Cation Family'].reset_index(drop=True)
# Calculate the standard deviation for each cluster
cluster_std = data.groupby('Cluster').std()

# Combine both the mean and the standard deviation for visualization
cluster_profile_with_std = cluster_profile.join(cluster_std, rsuffix='_std')

In [ ]:
import pandas as pd
import plotly.express as px
# import kaliedo
# Assuming 'data' is your DataFrame and it contains columns 'Cluster' and 'Cation Family'

# Add the 'Cation Family' column to 'data'


# Calculate the total number of points for each cluster
total_cluster_counts = data['Cluster'].value_counts().reset_index()
total_cluster_counts.columns = ['Cluster', 'Total_Count']

# Calculate the counts for each cation family within each cluster
cation_cluster_counts = data.groupby(['Cluster', 'Cation Family']).size().reset_index(name='Count')

# Merge to get the total counts for each cluster
cation_cluster_counts = cation_cluster_counts.merge(total_cluster_counts, on='Cluster')

# Calculate percentages within each cluster for cation families
cation_cluster_counts['Cation_Percentage'] = cation_cluster_counts['Count'] / cation_cluster_counts['Total_Count'] * 100

# Calculate overall cluster percentages
overall_cluster_percentage = data['Cluster'].value_counts(normalize=True).reset_index()
overall_cluster_percentage.columns = ['Cluster', 'Overall_Percentage']
overall_cluster_percentage['Overall_Percentage'] *= 100

# Merge to get overall cluster percentages
cation_cluster_counts = cation_cluster_counts.merge(overall_cluster_percentage, on='Cluster')

# Convert necessary columns to strings for concatenation
cation_cluster_counts['Cluster'] = cation_cluster_counts['Cluster'].astype(str)
cation_cluster_counts['Overall_Percentage'] = cation_cluster_counts['Overall_Percentage'].round(2).astype(str)
cation_cluster_counts['Cation Family'] = cation_cluster_counts['Cation Family'].astype(str)
cation_cluster_counts['Cation_Percentage'] = cation_cluster_counts['Cation_Percentage'].round(2).astype(str)

# Prepare the custom data for the inner and outer rings
cation_cluster_counts['Cluster_Custom'] = cation_cluster_counts['Cluster'] + '<br> ' + cation_cluster_counts['Overall_Percentage'] + '%'
cation_cluster_counts['Cation_Custom'] = cation_cluster_counts['Cation Family'] + '<br> ' + cation_cluster_counts['Cation_Percentage'] + '%'

# Set a single color for all rings
single_color = "skyblue"  # Choose your desired color

# Create a sunburst chart using Plotly
fig = px.sunburst(
    cation_cluster_counts,
    path=['Cluster_Custom', 'Cation_Custom'],
    values='Count',
    color='Cluster',  # Set color based on cluster for uniformity
    color_discrete_sequence=[single_color] * len(cation_cluster_counts),  # Set a single color for all segments
    # title='Distribution of Cation Families Within Each Cluster'
)

# Remove color influence from 'values' and add black boundaries
fig.update_traces(marker=dict(colors=[single_color] * len(cation_cluster_counts), 
                              line=dict(color='black', width=2)))  # Add black borders

# Update layout for better visualization
fig.update_layout(
    margin=dict(t=50, l=25, r=25, b=25),
    title={
        # 'text': "Distribution of B-Cation Families Within Each Cluster",
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {
            'family': "Arial, sans-serif",
            'size': 20,
            'color': "Black",
            'weight': 'bold'  # Make the title font bold
        }
    },
    font=dict(
        family="Arial, sans-serif",
        size=20,
        color="Black",
        weight='bold'  # Make the font bold
    ),
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    hoverlabel=dict(
        font_size=20,
        font_family="Arial"
    )
)

# Add hover information
fig.update_traces(hovertemplate="<b>%{label}</b><br>")


# Save the figure
fig.write_image('tsne_cation_distribution_sunburst.pdf', format='pdf', scale=10)  # Adjust scale for resolution

# Show the plot
fig.show()


In [ ]:
import pandas as pd
import plotly.express as px

# Assuming 'data' is your DataFrame and it contains columns 'Cluster' and 'Halide Group'

# Add the 'Halide Group' column to 'data'

# Calculate the total number of points for each cluster
total_cluster_counts = data['Cluster'].value_counts().reset_index()
total_cluster_counts.columns = ['Cluster', 'Total_Count']

# Calculate the counts for each halide group within each cluster
halide_cluster_counts = data.groupby(['Cluster', 'Halide Group']).size().reset_index(name='Count')

# Merge to get the total counts for each cluster
halide_cluster_counts = halide_cluster_counts.merge(total_cluster_counts, on='Cluster')

# Calculate percentages within each cluster for halide groups
halide_cluster_counts['Halide_Percentage'] = halide_cluster_counts['Count'] / halide_cluster_counts['Total_Count'] * 100

# Calculate overall cluster percentages
overall_cluster_percentage = data['Cluster'].value_counts(normalize=True).reset_index()
overall_cluster_percentage.columns = ['Cluster', 'Overall_Percentage']
overall_cluster_percentage['Overall_Percentage'] *= 100

# Merge to get overall cluster percentages
halide_cluster_counts = halide_cluster_counts.merge(overall_cluster_percentage, on='Cluster')

# Convert necessary columns to strings for concatenation
halide_cluster_counts['Cluster'] = halide_cluster_counts['Cluster'].astype(str)
halide_cluster_counts['Overall_Percentage'] = halide_cluster_counts['Overall_Percentage'].round(2).astype(str)
halide_cluster_counts['Halide Group'] = halide_cluster_counts['Halide Group'].astype(str)
halide_cluster_counts['Halide_Percentage'] = halide_cluster_counts['Halide_Percentage'].round(2).astype(str)

# Prepare the custom data for the inner and outer rings
halide_cluster_counts['Cluster_Custom'] = halide_cluster_counts['Cluster'] + '<br> ' + halide_cluster_counts['Overall_Percentage'] + '%'
halide_cluster_counts['Halide_Custom'] = halide_cluster_counts['Halide Group'] + '<br> ' + halide_cluster_counts['Halide_Percentage'] + '%'

# Set a single color for all rings
single_color = "lightgreen"  # Choose your desired color

# Create a sunburst chart using Plotly
fig = px.sunburst(
    halide_cluster_counts,
    path=['Cluster_Custom', 'Halide_Custom'],
    values='Count',
    color='Cluster',  # Set color based on cluster for uniformity
    color_discrete_sequence=[single_color] * len(halide_cluster_counts),  # Set a single color for all segments
    # title='Distribution of Halide Groups Within Each Cluster'
)

# Remove color influence from 'values' and add black boundaries
fig.update_traces(marker=dict(colors=[single_color] * len(halide_cluster_counts), 
                              line=dict(color='black', width=2)))  # Add black borders

# Update layout for better visualization
fig.update_layout(
    margin=dict(t=50, l=25, r=25, b=25),
    title={
        # 'text': "Distribution of Halide Groups Within Each Cluster",
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {
            'family': "Arial, sans-serif",
            'size': 20,
            'color': "Black",
            'weight': 'bold'  # Make the title font bold
        }
    },
    font=dict(
        family="Arial, sans-serif",
        size=20,
        color="Black",
        weight='bold'  # Make the font bold
    ),
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    hoverlabel=dict(
        font_size=20,
        font_family="Arial"
    )
)

# Add hover information
fig.update_traces(hovertemplate="<b>%{label}</b><br>")


# Save the figure
fig.write_image('tsne_halide_distribution_sunburst.pdf', format='pdf', scale=10)  # Adjust scale for resolution

# Show the plot
fig.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# === Step 1: Prepare the data ===

# Group by cluster and a_site_family to get counts
a_site_cluster_counts = data.groupby(['Cluster', 'a_site_family']).size().unstack(fill_value=0)

# Convert counts to percentages
a_site_cluster_percentages = a_site_cluster_counts.div(a_site_cluster_counts.sum(axis=1), axis=0) * 100

# Transpose: a_site_family becomes rows, clusters become columns
df = a_site_cluster_percentages.T

# Rename columns for clarity (e.g., Cluster 0 → Cluster 0)
df.columns = [f'Cluster {int(col)}' for col in df.columns]

# === Step 2: Sort a_site_family by increasing rA (Å) ===

# Ensure df2 has columns: a_site_family and rA
# Sort df2 by rA, get sorted a_site_family list
sorted_a_sites = data.sort_values(by='rA(Ang)')['a_site_family'].drop_duplicates().values

# Reorder rows of df to match increasing rA
df = df.loc[sorted_a_sites]

# === Step 3: Plot the heatmap ===

plt.figure(figsize=(12, 10))

sns.heatmap(
    df,
    annot=True,
    cmap='YlOrBr',
    fmt='.2f',
    cbar_kws={'label': 'Concentration (%)'},
    annot_kws={'size': 16, 'weight': 'bold'},
    xticklabels=df.columns,
    yticklabels=df.index
)

# Customize axes and ticks
plt.ylabel('', fontsize=16, fontweight='bold')
plt.xticks(rotation=45, ha='right', fontsize=16, fontweight='bold')
plt.yticks(fontsize=16, fontweight='bold')
plt.tight_layout()

# Customize the colorbar
cbar = plt.gca().collections[0].colorbar
cbar.set_label('Concentration (%)', fontsize=16, weight='bold', labelpad=20)
cbar.ax.tick_params(labelsize=16, width=2, direction='in', length=6, grid_color='gray', grid_alpha=0.5)

# Bold the colorbar tick labels
for label in cbar.ax.get_yticklabels():
    label.set_fontsize(16)
    label.set_fontweight('bold')

# === Step 4: Save the plot ===

plt.savefig('ion_cluster_heatmap.pdf', dpi=2000, bbox_inches='tight', transparent=True)


In [ ]:
cluster_profile.rename(columns={'Bandgap, GGA (eV)':'Bandgap','A SITE DFE':'E$_A$', 'B SITE DFE':'E$_B$', 'X SITE DFE':'E$_X$',
                   'Atomization energy (eV/atom)':'E$_{atomization}$', 'Relative energy1 (eV/atom)':'E$_{relative1}$',
                   'Relative energy2 (eV/atom)':'E$_{relative2}$','Density (g/cm^3)':'Density','rA(Ang)':'r$_A$','rB (Ang)':'r$_B$','rX(Ang)':'r$_X$','octahedral factor':'Octahedral factor','Volume of the unit cell (A^3)':'Unit cell volume', 'Dielectric constant, total':'Dielectric constant'},inplace=True)
cluster_std.rename(columns={'Bandgap, GGA (eV)':'Bandgap','A SITE DFE':'E$_A$', 'B SITE DFE':'E$_B$', 'X SITE DFE':'E$_X$',
                   'Atomization energy (eV/atom)':'E$_{atomization}$', 'Relative energy1 (eV/atom)':'E$_{relative1}$',
                   'Relative energy2 (eV/atom)':'E$_{relative2}$','Density (g/cm^3)':'Density','rA(Ang)':'r$_A$','rB (Ang)':'r$_B$','rX(Ang)':'r$_X$','octahedral factor':'Octahedral factor','Volume of the unit cell (A^3)':'Unit cell volume', 'Dielectric constant, total':'Dielectric constant'},inplace=True)

In [ ]:
# cluster_profile.rename(columns={'Bandgap, GGA (eV)':'Bandgap'},inplace=True)
features = ['Bandgap', 'Dielectric constant','Refractive index', 'Density', 'Unit cell volume','E$_{atomization}$', 'E$_A$', 'E$_B$', 'E$_X$']

# Plotting the cluster profiles with standard deviation
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(16, 16))
axes = axes.flatten()
plt.rcParams.update({'font.size': 12, 'font.weight': 'bold'})

for i, feature in enumerate(features):
    # Plot the mean value
    sns.barplot(x=cluster_profile.index, y=cluster_profile[feature], ax=axes[i], color='blue', label='Mean')
    
    # Plot the standard deviation as error bars
    axes[i].errorbar(cluster_profile.index, cluster_profile[feature], yerr=cluster_std[feature], fmt='none', color='black', capsize=5)
    
    # axes[i].set_title(f'{feature}', fontsize=14, fontweight='bold')
    axes[i].set_xlabel('Cluster', fontsize=14, fontweight='bold')
    axes[i].set_ylabel(feature, fontsize=14, fontweight='bold')
    plt.xticks(rotation=0, fontsize=12, fontweight='bold')  # Bold x-axis labels
    plt.yticks(rotation=0, fontsize=12, fontweight='bold')  # Bold y-axis labels

plt.tight_layout()
plt.savefig('tsne_kmeans_cluster-profiles_with_std.pdf', format='pdf', dpi=2000)
plt.show()